## Import

In [1]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np
import cv2

from modules.prediction_image import resize_image, evaluate, stackImages, getContours, empty

model = load_model('model.h5')

## Test du modele

In [2]:
# model = load_model('model.h5')

# labels_array = ['apple', 'bee']
# choix = [
#     'assets/apple/0001.png'
#     , 'assets/apple/0002.png'
#     , 'assets/apple/0003.png'
#     , 'assets/apple/0004.png'
#     , 'assets/apple/0005.png'
#     , 'assets/bee/0001.png'
#     , 'assets/bee/0002.png'
#     , 'assets/bee/0003.png'
#     , 'assets/bee/0004.png'
#     , 'assets/bee/0005.png'
# ]

# for c in choix:
#     test_image = image.load_img(c, target_size = (32, 32))
#     test_image = image.img_to_array(test_image)
#     test_image = np.expand_dims(test_image, axis = 0)
#     result = model.predict(test_image)

#     preds = model.predict_classes(test_image)
#     prob = model.predict_proba(test_image)

#     index = preds[0]
#     img = c.split('/')[-1]
#     print(f'Img {img}. Cet objet est un(e) "{labels_array[index]}".')

#     prob_sort = np.argsort(prob[0])[-2:]

#     for ps in prob_sort:
#         proba = prob[0][ps]
#         label = labels_array[ps]
#         print(f'{label}:\t', round(proba, 3) )

#     print()

## Detection d'objet avec openCV

In [3]:
path = "assets/image_internet/"

img_originale = cv2.imread(path + "pomme_abeille.jpg")

Threshold1 = 80
Threshold2 = 80
AreaMin = 850
AreaMax = 40000

In [38]:
img_originale = cv2.imread(path + "pomme_abeille.jpg")

from math import sqrt

print(img_originale.shape)
area = img_originale.shape[0] * img_originale.shape[1]
ratio = sqrt( (800*800) / area )
print(ratio)
dim = round(img_originale.shape[1] * ratio), round(img_originale.shape[0] * ratio)
print(dim)
# img_originale = cv2.resize(img_originale, dim)
# print(img_originale.shape)

# cv2.imshow("image", img_originale)
# cv2.waitKey()
# cv2.destroyWindow("image")

# img_originale.shape[0] * img_originale.shape[1]

(1390, 1300, 3)
8.0
(10400, 11120)


In [7]:
img_traite = img_originale.copy()

imgBlur = cv2.GaussianBlur(img_traite, (9, 9), 1)
imgCanny = cv2.Canny(img_traite, Threshold1, Threshold2)
kernel = np.ones((5, 5))
imgDil = cv2.dilate(imgCanny, kernel, iterations=1)

contours = cv2.findContours(imgCanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]
contours = [cnt for cnt in contours if cv2.contourArea(cnt) > AreaMin and cv2.contourArea(cnt) < AreaMax]

img_affichage = img_traite.copy()

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)

    img = img_traite[y:y+h, x:x+w]

    img = resize_image(img)

    predictions = list()
    label = evaluate(model, img)

    cv2.putText(img_affichage, label, (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
    cv2.rectangle(img_affichage, (x , y ), (x + w , y + h ), (0, 0, 255), 5)


cv2.imshow("image", img_affichage)
cv2.waitKey()
cv2.destroyWindow("image")

## Reglages sur l'image

In [6]:
cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters", 640, 240)
cv2.createTrackbar("Threshold1", "Parameters",   Threshold1, 255, empty)
cv2.createTrackbar("Threshold2", "Parameters",   Threshold2, 255, empty)
cv2.createTrackbar("AreaMin", "Parameters",      AreaMin, 3000, empty)
cv2.createTrackbar("AreaMax", "Parameters",      AreaMax, 50000, empty)

img = img_originale

while True:

    imgContour = img.copy()
    imgBlur = cv2.GaussianBlur(img, (9, 9), 1)
    threshold1 = cv2.getTrackbarPos("Threshold1", "Parameters")
    threshold2 = cv2.getTrackbarPos("Threshold2", "Parameters")
    imgCanny = cv2.Canny(imgContour, threshold1, threshold2)
    kernel = np.ones((5, 5))
    imgDil = cv2.dilate(imgCanny, kernel, iterations=1)
    getContours(imgCanny, imgContour)
    imgStack = stackImages(0.8, ([img,imgCanny], [imgDil,imgContour]) )

    cv2.imshow("Result", imgStack)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()